In [ ]:
#r "nuget:DataBoss.DataPackage, 0.0.93"
#r "nuget:DataBoss.Data, 0.0.93"
//standard includes
using DataBoss.Data;
using DataBoss.DataPackage;

Installing package DataBoss.DataPackage, version 0.0.93...

Installing package DataBoss.Data, version 0.0.93...

_visualization functions, you can safely skip this cell_

In [ ]:
using System.Data;
using Microsoft.DotNet.Interactive.Formatting;
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;

static void Dump(DataPackage dp, int? limit = null) {
    var rs = dp.Resources
        .Where(x => x.Format == "csv")
        .Select(x => (Heading: th(x.Name), Result: td(ReadAsHtmlTable(x.Read(), limit: limit))))
        .ToList();

    display(table(
        thead(rs.Select(x => x.Heading)),
        tbody(rs.Select(x => x.Result))
    ));
}

static object Dump(DataBoss.DataPackage.TabularDataResource xs, int? limit = null) => Dump(xs.Read(), limit: limit);

static object Dump(IDataReader xs, int? limit = null) => display(ReadAsHtmlTable(xs, limit: limit));

static object ReadAsHtmlTable(IDataReader xs, int? limit = null) {
    try {
        limit ??= int.MaxValue;
        var rows = new List<object>();
        for(var i = 0;xs.Read() && i < limit; ++i)
            rows.Add(Enumerable.Range(0, xs.FieldCount).Select(x => td(xs.GetValue(x))).ToList());

        return table(
            thead(Enumerable.Range(0, xs.FieldCount).Select(x => th[style:"font-weight:bold"](xs.GetName(x)))),
            tbody(rows.Select(x => tr(x))));
    } finally {
        xs.Dispose();
    } 
}


# Defining a simple resource

In [ ]:
var dp = new DataPackage();

dp.AddResource(xs => xs.WithName("numbers").WithData(Enumerable.Range(0, 2).Select(x => new { Value = x })));

Dump(dp);

0
1


**DataPackage.Load** supports directory paths containing a datapackage.json, zip files and http.

In [ ]:
var countries = DataPackage.Load(@"https://datahub.io/core/country-list/r/country-list_zip.zip");

In [ ]:
Dump(countries.GetResource("data_csv"), limit: 10);

Afghanistan,AF
Åland Islands,AX
Albania,AL
Algeria,DZ
American Samoa,AS
Andorra,AD
Angola,AO
Anguilla,AI
Antarctica,AQ
Antigua and Barbuda,AG


# Resource (DataReader) Transformation

In [ ]:
var c2 = new DataPackage();

c2.AddResource(countries.GetResource("data_csv"));
c2.UpdateResource("data_csv", xs => xs
    .WithName("countries") //resource can be renamed.
    .Transform(x => 
    {
        var id = 0;
        x.Transform("Code", (string value) => value.ToLower()); //typed transform
        x.Add(0, "Id", r => ++id); //columns can be added at any existing ordinal
        x.Transform("Name", (string value) => value.ToUpper());
        x.Add("NameLen", r => r["Name"].ToString().Length); //record based
        x.Add("Source", r => $"{r.Source["Name"]} ({r.Source["Code"]})"); //from non transformed source
    })
);
Dump(c2, limit: 10);

1,AFGHANISTAN,af,11,Afghanistan (AF)
2,ÅLAND ISLANDS,ax,13,Åland Islands (AX)
3,ALBANIA,al,7,Albania (AL)
4,ALGERIA,dz,7,Algeria (DZ)
5,AMERICAN SAMOA,as,14,American Samoa (AS)
6,ANDORRA,ad,7,Andorra (AD)
7,ANGOLA,ao,6,Angola (AO)
8,ANGUILLA,ai,8,Anguilla (AI)
9,ANTARCTICA,aq,10,Antarctica (AQ)
10,ANTIGUA AND BARBUDA,ag,19,Antigua and Barbuda (AG)


# Creating Resrouces Incrementally

In [ ]:
var n = 0;
var numbers = Enumerable.Range(0, 3).Select(x => new { Value = ++n });
var myNumbers  = new DataPackage();

void AddOrAppend<T>(DataPackage dp, string name, IEnumerable<T> rows) {
    dp.AddOrUpdateResource(name, xs => xs.WithData(rows), xs => xs.WithData(() => xs.Read().Concat(rows)));
}

AddOrAppend(myNumbers, "numbers", numbers.ToList());
Dump(myNumbers);
Console.WriteLine("---[ Update ]---");
AddOrAppend(myNumbers, "numbers", numbers.ToList());
Dump(myNumbers);


1
2
3


---[ Update ]---


1
2
3
4
5
6
